In [1]:
print("hello world")

hello world


In [9]:
from PIL import Image
from io import BytesIO
import httpx
import asyncio
from requests_html import AsyncHTMLSession
import os

async def extract_product_images(url):
    session = AsyncHTMLSession()
    response = await session.get(url)
    html = response.html.html
    
    # find the JSON-LD script containing product information
    jsonld_script = None
    for script in html.split("</script>"):
        if "@type\":\"Product\"" in script:
            jsonld_script = script + "</script>"
            break
    if jsonld_script is None:
        return [] # no product JSON-LD script found
    # parse the JSON-LD script to extract image URLs
    links = []
    start_index = jsonld_script.find("\"image\":[{") + len("\"image\":[{")
    end_index = jsonld_script.find("}]", start_index) + len("}]")
    image_str = jsonld_script[start_index:end_index]
    while True:
        url_start = image_str.find("\"contentUrl\":\"") + len("\"contentUrl\":\"")
        if url_start == len("\"contentUrl\":\"") - 1:
            break
        url_end = image_str.find("\"", url_start)
        url = image_str[url_start:url_end]
        if url.endswith(".jpg"):
            # print(url)
            links.append(url)
        image_str = image_str[url_end:]
    # print(links)
    start_index = jsonld_script.find("\"name\":\"") + len("\"name\":\"")
    end_index = jsonld_script.find("\"", start_index)
    folder_name = jsonld_script[start_index:end_index].replace(" ", "_").replace("-", "_")
    ###########
    os.makedirs(os.path.join(os.getcwd(), "clothing"), exist_ok=True)
    os.makedirs(os.path.join(os.getcwd(), "clothing", folder_name), exist_ok=True)

    for url in links:
        filename = url.split("/")[-1]
        client = httpx.AsyncClient(http2=True)
        response = await client.get(url)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image.save(
            os.path.join(os.getcwd(), "clothing", folder_name, filename), format="JPEG"
        )
    print(folder_name, '....done')


if __name__ == "__main__":
    url_list=(
        "https://www.farfetch.com/ae/shopping/men/versace-striped-loose-fit-shirt-item-18835459.aspx",
        # "https://www.farfetch.com/au/shopping/women/oscar-de-la-renta-floral-applique-crochet-knit-dress-item-19028843.aspx",
        # "https://www.farfetch.com/au/shopping/women/zuhair-murad-floral-lace-peplum-blouse-item-18394400.aspx",
        # "https://www.farfetch.com/au/shopping/women/miu-miu-sequin-embellished-cropped-jacket-item-18252949.aspx",
        # "https://www.farfetch.com/au/shopping/women/valentino-tulle-illusione-sequin-embroidered-blouse-item-19183134.aspx",
        # "https://www.farfetch.com/au/shopping/women/oscar-de-la-renta-floral-embroidery-asymmetric-top-item-14595708.aspx",
        # "https://www.farfetch.com/au/shopping/women/oscar-de-la-renta-fil-coupe-bustier-top-item-14333843.aspx",
        # "https://www.farfetch.com/au/shopping/women/elie-saab-feather-trim-crepe-blouse-item-19116639.aspx",
        # "https://www.farfetch.com/au/shopping/women/oscar-de-la-renta-floral-tapestry-ruched-hem-dress-item-18658945.aspx?storeid=11092&rtype=inspire_portal_pdp_generic_a&rpos=3&rid=8b325145-a03f-4d34-8bfa-0c2efc24090b",
        # "https://www.farfetch.com/au/shopping/women/oscar-de-la-renta-floral-lace-a-line-gown-item-19029712.aspx?storeid=10351&rtype=inspire_portal_pdp_generic_a&rpos=2&rid=8b325145-a03f-4d34-8bfa-0c2efc24090b",
        # "https://www.farfetch.com/au/shopping/women/oscar-de-la-renta-floral-jacquard-gown-item-19029714.aspx?storeid=11092&rtype=inspire_portal_pdp_generic_a&rpos=1&rid=8b325145-a03f-4d34-8bfa-0c2efc24090b",
        # "https://www.farfetch.com/au/shopping/women/balmain-sky-print-tulle-maxi-dress-item-19684854.aspx",
        # "https://www.farfetch.com/au/shopping/women/ferragamo-printed-draped-midi-dress-item-19709156.aspx",
        # "https://www.farfetch.com/au/shopping/women/ferragamo-graphic-print-silk-shirt-item-18885407.aspx?storeid=13539",
        # "https://www.farfetch.com/au/shopping/women/ferragamo-rhinestone-embellished-straight-leg-trousers-item-19276278.aspx?storeid=14540",
        # "https://www.farfetch.com/au/shopping/women/ferragamo-leather-hooded-jacket-item-19709178.aspx?storeid=14540",
        # "https://www.farfetch.com/au/shopping/women/ferragamo-transparent-drape-dress-item-19709168.aspx?storeid=14540",
        # "https://www.farfetch.com/au/shopping/women/ferragamo-one-shoulder-draped-dress-item-19709185.aspx?storeid=14540",
        # "https://www.farfetch.com/au/shopping/women/ferragamo-panelled-herringbone-pattern-single-breasted-coat-item-19590158.aspx?storeid=10344",
        # "https://www.farfetch.com/au/shopping/women/brunello-cucinelli-floral-wrap-maxi-skirt-item-19244730.aspx",
        # "https://www.farfetch.com/au/shopping/women/dolce-gabbana-crystal-embellished-wrap-around-swimsuit-item-17734776.aspx",
        # "https://www.farfetch.com/au/shopping/women/dolce-gabbana-carretto-print-long-chiffon-dress-item-16924802.aspx",
        # "https://www.farfetch.com/au/shopping/women/amir-slama-floral-print-silk-long-dress-item-14847295.aspx",
        # "https://www.farfetch.com/au/shopping/women/philipp-plein-crystal-studded-jacket-item-18871207.aspx",
        # "https://www.farfetch.com/au/shopping/women/philipp-plein-studded-leather-biker-jacket-item-15413095.aspx",
        # "https://www.farfetch.com/au/shopping/women/oscar-de-la-renta-butterfly-applique-cropped-jacket-item-19466557.aspx",
        # "https://www.farfetch.com/au/shopping/women/brunello-cucinelli-monili-tab-embellished-shearling-jacket-item-18610122.aspx",
        # "https://www.farfetch.com/au/shopping/women/saiid-kobeisy-bead-embellished-peplum-suit-item-16869458.aspx",
        # "https://www.farfetch.com/au/shopping/women/valentino-tulle-illusione-embroidered-blouse-item-18656968.aspx",
        # "https://www.farfetch.com/au/shopping/women/etro-sequin-embellished-plunge-cropped-top-item-19187728.aspx",
        # "https://www.farfetch.com/au/shopping/women/oscar-de-la-renta-floral-embroidery-asymmetric-top-item-14595708.aspx",
        # "https://www.farfetch.com/au/shopping/women/valentino-golden-wings-brocade-overshirt-item-19184246.aspx",
        # "https://www.farfetch.com/au/shopping/women/dolce-gabbana-kim-dolcegabbana-beaded-fringe-bustier-top-item-19391177.aspx",
        # "https://www.farfetch.com/au/shopping/women/valentino-bead-embellished-cotton-poplin-shirt-item-19183299.aspx",
        # "https://www.farfetch.com/au/shopping/women/oscar-de-la-renta-crystal-embellished-silk-blouse-item-19303249.aspx",
        # "https://www.farfetch.com/au/shopping/women/ferragamo-suede-regular-trousers-item-19662679.aspx",
        # "https://www.farfetch.com/au/shopping/women/ferragamo-jacquard-double-breast-coat-item-19568157.aspx",
        # "https://www.farfetch.com/au/shopping/women/ferragamo-sleeveless-graphic-print-dress-item-18885410.aspx",
        # "https://www.farfetch.com/au/shopping/women/ferragamo-graphic-print-silk-midi-skirt-item-18885406.aspx",
        # "https://www.farfetch.com/au/shopping/women/ferragamo-logo-motif-virgin-wool-cardigan-item-18528699.aspx"
    )
    
    for url in url_list:
        print(url)
        await extract_product_images(url=url)


https://www.farfetch.com/ae/shopping/men/versace-striped-loose-fit-shirt-item-18835459.aspx
striped_loose_fit_shirt ....done


In [2]:
from PIL import Image
from io import BytesIO
import httpx
import asyncio
from requests_html import AsyncHTMLSession
import os

# async def scrape(url):
#     session = AsyncHTMLSession()
#     r = await session.get(url)
#     image_links = r.html.find("img")

#     links = []
#     for imag in image_links:
#         try:
#             link = imag.attrs["src"]
#             if link.startswith("https://cdn-image"):
#                 links.append(link)
#         except KeyError:
#             pass
#     folder_name = url.split("/")[-1].split(".")[0].replace("-", "_")
#     os.makedirs(os.path.join(os.getcwd(), "clothing"), exist_ok=True)
#     os.makedirs(os.path.join(os.getcwd(), "clothing", folder_name), exist_ok=True)

#     for url in links:
#         filename = url.split("/")[-1]
#         client = httpx.AsyncClient(http2=True)
#         response = await client.get(url)
#         image = Image.open(BytesIO(response.content)).convert("RGB")
#         image.save(
#             os.path.join(os.getcwd(), "clothing", folder_name, filename), format="JPEG"
#         )


# if __name__ == "__main__":
#     url = "https://www.farfetch.com/ae/shopping/men/versace-striped-loose-fit-shirt-item-18835459.aspx"
#     await scrape(url=url)


In [3]:
from PIL import Image

In [23]:
x = Image.open('/home/ali/siavash/scraper/farfetch/722473_Z6519_4440_001_100_0000_Light-Light-cashmere-twill-formal-jacket.jpg')


In [24]:
x.mode

'RGB'

In [ ]:
!pip install httpx[http2]
!pip install requests-html